# dog-breed-prediction

=> In this project, we will see how to use Keras and TensorFlow to build, train, and test a Convolutional Neural Network capable of identifying the breed of a dog in a supplied image. This is a supervised learning problem, specifically a multiclass classification problem.

=> convolutional neural network r generly used for img classification.

In [ ]:
# step to solve the problem
1) first step load data from kaggle.
2) after that load the csv for label that contain img I.D and breed.
3) after will check those breed counts
4) after will perform one hot encoding on labels data prediction column.
5) then after will load the image,convert them to an array and normalize them.
6) then will check shape and size of X and Y  data.
7) then build the model network architecture.
8) split the data and fit in into the model and create an accuracy plot.
9) evaluate the model for accuracy score.
10) using the model for prediction.

##### in this project, we will start by downloading the dataset creating the model and finding out prediction using model.
##### this model can be optimized diff hyperparameter using this model for higher accuracy.
##### this model can be used to predict diff breed of dogs which can be further used by diff NGOs working on save animal and for education purpose also.

# in this project we load the dataset from kaggle 

=> we load the data from kaggle API through

In [1]:
# for that firstly we install the Kaggle API client.

!pip install kaggle

  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73051 sha256=06648c4defe61fde755216c0b77c8485975cdbe0ef0105347867d8f4231018bd
  Stored in directory: c:\users\lenovo\appdata\local\pip\cache\wheels\ac\b2\c3\fa4706d469b5879105991d1c8be9a3c2ef329ba9fe2ce5085e
Successfully built kaggle


In [2]:
# The Kaggle API client expects this file to be create kaggle file to store in json file
!mkdir .kaggle

In [3]:
# now check out kaggle json file

!cd C:\Users\Lenovo\DATA SCIENCE PROJECT python file\2) DOG-BREED-PREDICTION && dir

 Volume in drive C has no label.
 Volume Serial Number is 8CA7-EC90

 Directory of C:\Users\Lenovo\DATA SCIENCE PROJECT python file\2) DOG-BREED-PREDICTION

02/22/2022  11:19 AM    <DIR>          .
02/22/2022  11:19 AM    <DIR>          ..
02/22/2022  12:48 AM    <DIR>          .ipynb_checkpoints
02/22/2022  11:20 AM    <DIR>          .kaggle
02/22/2022  11:19 AM            30,492 DOG-BREED.ipynb
02/22/2022  12:49 AM    <DIR>          dog_data
               1 File(s)         30,492 bytes
               5 Dir(s)  75,897,098,240 bytes free


# Next To store the data we will create a new directory and make it as current working directory.

In [4]:
# # Creating directory and changing the current working directory

!mkdir dog_data
%cd dog_data

C:\Users\Lenovo\DATA SCIENCE PROJECT python file\2) DOG-BREED-PREDICTION\dog_data


A subdirectory or file dog_data already exists.


## Searching Kaggle for the required dataset using search option(-s) with title 'dogbreedidfromcomp'. 

=>We can also use different search options like searching competitions, notebooks, kernels, datasets, etc.

In [5]:
# Searching for dataset
!kaggle datasets list -s dog-breed-id-from-comp

ref                                title                    size  lastUpdated          downloadCount  voteCount  usabilityRating  
---------------------------------  ----------------------  -----  -------------------  -------------  ---------  ---------------  
catherinehorng/dogbreedidfromcomp  dog-breed-id-from-comp  691MB  2020-06-26 03:09:05            831          3  0.1764706        


## After searching the data next step would be downloading the data into jupyter notebook using references found in search option.

In [6]:
# Downloading dataset and coming out of directory
!kaggle datasets download catherinehorng/dogbreedidfromcomp

dogbreedidfromcomp.zip: Skipping, found more recently modified local copy (use --force to force download)


## We will unzip the data which is downloaded and remove the irrelevant files

In [7]:
# Unzipping downloaded file and removing unusable file

import zipfile
with zipfile.ZipFile("dogbreedidfromcomp.zip","r") as zip_ref:
    zip_ref.extractall()

# now our data analysis part start

### Important library imports

In [9]:
# Important library imports
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

from tqdm import tqdm
from keras.preprocessing import image
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam

In [11]:
# we load the csv file for that we create the directory path

import os
os.chdir(r"C:\Users\Lenovo\DATA SCIENCE PROJECT python file\2) DOG-BREED-PREDICTION\dog_data")
os.getcwd()

'C:\\Users\\Lenovo\\DATA SCIENCE PROJECT python file\\2) DOG-BREED-PREDICTION\\dog_data'

### Loading the labels data into dataframe and viewing it. Here we analysed that labels contains 10222 rows and 2 columns.

In [12]:
# Read the labels.csv file and checking shape and records
labels_all = pd.read_csv('labels.csv')
print(labels_all.shape)
labels_all.head(10)

(10222, 2)


,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever
5,002211c81b498ef88e1b40b9abf84e1d,bedlington_terrier
6,00290d3e1fdd27226ba27a8ce248ce85,bedlington_terrier
7,002a283a315af96eaea0e28e7163b21b,borzoi
8,003df8b8a8b05244b1d920bb6cf451f9,basenji
9,0042188c895a2f14ef64a918ed9c7b64,scottish_deerhound


### Here we are finding out the count per class i.e. total data in each class using value_counts() function.

In [ ]:
# lets visualize the number of each group that is the value of each breed.

In [13]:
# Visualize the number of each breeds
breeds_all = labels_all["breed"]
breed_counts = breeds_all.value_counts()
breed_counts.head()

scottish_deerhound      126
maltese_dog             117
afghan_hound            116
entlebucher             115
bernese_mountain_dog    114
Name: breed, dtype: int64

### We will work on only 3 breeds due to limited computational power. You can consider more classes as per your system computational power

In [ ]:
# so the totol number each class can be find,not using the commerce function as the dataset is more large.
# and due to computation power less computation power.so what we will do is ?
# so we select only three breed for classification.

In [14]:
# Selecting first 3 breeds (Limitation due to computation power)
CLASS_NAMES = ['scottish_deerhound','maltese_dog','bernese_mountain_dog']
labels = labels_all[(labels_all['breed'].isin(CLASS_NAMES))]
labels = labels.reset_index()
labels.head()

,index,id,breed
0,9,0042188c895a2f14ef64a918ed9c7b64,scottish_deerhound
1,12,00693b8bc2470375cc744a6391d397ec,maltese_dog
2,79,01e787576c003930f96c966f9c3e1d44,scottish_deerhound
3,90,022b34fd8734b39995a9f38a4f3e7b6b,maltese_dog
4,118,02d54f0dfb40038765e838459ae8c956,bernese_mountain_dog


In [ ]:
# above output explanation:so this is our data, according to the classes of the dogs we have selected,
# will work on only three clasese tht is score test at home.

In [ ]:
# so next this is data is classification dataset,so we need to learn how to encode the target value that is classes of dogs

### As we are working with the classification dataset first we need to one hot encode the target value i.e. the classes. 

### After that we will read images and convert them into numpy array and finally normalizing the array.

In [19]:
# Creating numpy matrix with zeros
X_data = np.zeros((len(labels), 224, 224, 3), dtype='float32')
# One hot encoding
Y_data = label_binarize(labels['breed'], classes = CLASS_NAMES)

# Reading and converting image to numpy array bcz we need to normalizing dataset them.
for i in tqdm(range(len(labels))):
    img = image.load_img(r'C:\Users\lenovo\DATA SCIENCE PROJECT python file\2) DOG-BREED-PREDICTION\dog_data\train\%s.jpg' % labels['id'][i], target_size=(224, 224))
    img = image.img_to_array(img)
    x = np.expand_dims(img.copy(), axis=0)
    X_data[i] = x / 255.0   # this img pixcel from 0 to 255,so in order to normalize an img,we div tht img,so we div 255
    
# Printing train image and one hot encode shape & size
print('\nTrain Images shape: ',X_data.shape,' size: {:,}'.format(X_data.size))
print('One-hot encoded output shape: ',Y_data.shape,' size: {:,}'.format(Y_data.size))

100%|████████████████████████████████████████████████████████████████████████████████| 357/357 [00:04<00:00, 79.80it/s]


Train Images shape:  (357, 224, 224, 3)  size: 53,738,496
One-hot encoded output shape:  (357, 3)  size: 1,071


### Next we will create a network architecture for the model. 

### We have used different types of layers according to their features namely Conv_2d (It is used to create a convolutional kernel that is convolved with the input layer to produce the output tensor), max_pooling2d (It is a downsampling technique which takes out the maximum value over the window defined by poolsize), flatten (It flattens the input and creates a 1D output), Dense (Dense layer produce the output as the dot product of input and kernel).

### After defining the network architecture we found out the total parameters as 162,619.

In [16]:
# Building the Model
model = Sequential()

model.add(Conv2D(filters = 64, kernel_size = (5,5), activation ='relu', input_shape = (224,224,3)))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters = 32, kernel_size = (3,3), activation ='relu', kernel_regularizer = 'l2'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters = 16, kernel_size = (7,7), activation ='relu', kernel_regularizer = 'l2'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters = 8, kernel_size = (5,5), activation ='relu', kernel_regularizer = 'l2'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(128, activation = "relu", kernel_regularizer = 'l2'))
model.add(Dense(64, activation = "relu", kernel_regularizer = 'l2'))
model.add(Dense(len(CLASS_NAMES), activation = "softmax"))

model.compile(loss = 'categorical_crossentropy', optimizer = Adam(0.0001),metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 220, 220, 64)      4864      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 110, 110, 64)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 108, 108, 32)      18464     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 48, 48, 16)        25104     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 24, 24, 16)       0

### After defining the network architecture we will start with splitting the test and train data then dividing train data in train and validation data

In [17]:
# Splitting the data set into training and testing data sets
X_train_and_val, X_test, Y_train_and_val, Y_test = train_test_split(X_data, Y_data, test_size = 0.1)
# Splitting the training data set into training and validation data sets
X_train, X_val, Y_train, Y_val = train_test_split(X_train_and_val, Y_train_and_val, test_size = 0.2)

### Now we will train our model on 100 epochs and a batch size of 128. You can try using more number of epochs to increase accuracy. During each epochs we can see how the model is performing by viewing the training and validation accuracy

In [ ]:
# Training the model
epochs = 100
batch_size = 128

history = model.fit(X_train, Y_train, batch_size = batch_size, epochs = epochs, validation_data = (X_val, Y_val))

Epoch 1/50


### Here we analyse how the model is learning with each epoch in terms of accuracy.

In [ ]:
# Plot the training history
plt.figure(figsize=(12, 5))
plt.plot(history.history['accuracy'], color='r')
plt.plot(history.history['val_accuracy'], color='b')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'val'])

plt.show()

# We will use predict function to make predictions using this model also we are finding out the accuracy on the test set.

In [ ]:
Y_pred = model.predict(X_test)
score = model.evaluate(X_test, Y_test)
print('Accuracy over the test set: \n ', round((score[1]*100), 2), '%')

# Here you can see image with its original and predicted label.

In [ ]:
# Plotting image to compare
plt.imshow(X_test[1,:,:,:])
plt.show()

# Finding max value from predition list and comaparing original value vs predicted
print("Originally : ",labels['breed'][np.argmax(Y_test[1])])
print("Predicted : ",labels['breed'][np.argmax(Y_pred[1])])

# Conclusion:
We started with downloading the dataset creating the model and finding out the predictions using the model. We can optimize different hyper parameters in order to tune this model for a higher accuracy. This model can be used to predict different breeds of dogs which can be further used by different NGO's working on saving animals and for educational purposes also.

In [ ]:
# other step for doenload data from kaggle 

In [1]:
import opendatasets as od

In [2]:
datasets = "https://drive.google.com/drive/folders/11o5Nfj8kFd0xfkLsCrJrWI0CpBudveHq?usp=sharing"

In [ ]:
od.download('datasets')

In [ ]:
import os

In [ ]:
data_dir = '.\dog-breed-classification'

In [ ]:
os.listdir(data_dir)